### 第一章 Node简介

一个Node进程只允许一个线程处理js。一个大型网络应用，可以由多个Node进程通过通信来协作构成，每个Node进程都构成这个应用中的一个节点，这是Node名字的本意。

js作为一门图灵完备的语言，长久以来被禁锢在浏览器中，和其他语言一样，js的能力取决于浏览器中间层提供的支持有多少。浏览器的下层是操作系统，网卡，硬盘，显卡。。。

程序Node和chrome是对等物。他们实现的都是基于事件轮询的异步架构。

chrome的事件操作view/dom，Node的事件操作io。

chrome中大量计算会导致无法即时操作dom，界面卡顿。Node中大量计算会导致io设备的输入要排队输入和输出给io设备的内容要排队输出。解决方案是主线程排队处理io和dom，计算外包给其他线程，chromew是worker线程。Node是child_process。

在多个独立的事件中，如何沟通信息

单线程的最大好处是不用像多线程编程那样处处在意“状态”同步，不用锁。没有线程上下文的切换。

Node内部基于libuv实现跨平台。libuv称为平台层，在Node程序中，在操作系统的上层。libuv处理事件循环/文件操作不同操作系统的实现

### 第二章 模块机制

文件 === 模块 === 函数 === 对象。

每个模块被包装成函数，函数以exports,require,module,__filename,__dirname为输入。exports就是module.exports。通过module.paths读出所有可读的node_modules目录。该函数被runInThisContext调用。

Node导入模块，第一次导入时，会调用该模块并缓存返回对象，第二次导入时，Node直接返回该对象。用户模块返回对象缓存在Module._cache，以文件路径为key。库模块返回对象缓存在NativeModule._cache。

Node由commonjs和es规范。commonjs涵盖模块系统/二进制处理/Buffer/字符集编码/流/进程调用/文件系统fs/套接字/单元测试/包管理。commonjs规范让js具备开发大型应用比如服务器程序/客户端程序/命令行程序的能力。commonjs和cmd是两码事。

引入模块不指定文件类型，Node会以不同文件类型(js/json/node/目录)查找该文件。其他后缀类型的文件以js方式处理。如果查找到目录，会查找该目录的package.json的main属性指定的文件，若没有，则以目录内的index.js/index.json/index.node文件作为该目录的入口文件。

库模块除了js模块，还存在使用c++的模块（如fs/buffer/os模块）。库模块中的c++源码，在Node被编译成二进制时，一起被编译。
库模块的js源码放在Node项目的lib目录，库模块的c++源码放在src中。在编译c++前，会把lib的js源码放入c++的字符串数组，借助js2c.py。引入js库模块时，从内存中取得js代码字符串，也是包装成函数后调用。执行完后缓存在NativeModule._cache对象中。与用户模块的区别是，用户模块从硬盘中取代码字符串。

c++用户模块先编译为node文件（node文件 === windows下的dll文件 === linux的so文件）。通过执行node-gyp configure和node-gyp build编译得到node文件。引入node文件，即引入c++用户模块，底层调用dlopen函数。

Node项目通过gyp编译。linux g++/gcc === window vc++

安装本地包 npm i <含package.json的文件夹>，安装远程包npm i url。package.json的scripts用来定义钩子函数，例如key为install，在npm install时自动执行

### 第三章 异步IO

io输入 === 把数据从io设备带到内存 === 向cpu输入数据。阻塞，是实现同步效果的方式。

客户端，发起请求端。服务端，响应请求端。Node可以做客户端也可以做服务端。

代码同步：代码执行的顺序有一定的约束，比如后一行的代码一定在前一行的代码执行完之后才开始执行。换句话说，前一行代码阻塞后一行代码，是一种代码同步。异步则没有这样的同步约束。异步 === 不存在相互约束。同步 === 存在相互约束。
在js中调用网络io的同时可以调用其他函数，这种网络io是异步的。

从硬件结构上，计算机可以在同一时刻进行io操作和cpu计算，即程序可以在执行io api的同时执行计算。但出于方便易懂的考虑，程序员往往使用操作系统暴露的“在执行io操作时不执行计算”的io api，即阻塞型api。

linux用文件表示io设备。应用程序进行io操作：获取操作对象（文件符）并在操作对象上调用io方法。文件符是操作系统暴露给应用程序的操作对象。

操作系统分别提供了阻塞型io api和非阻塞型io api。调用阻塞型io api的应用程序可以拿到cpu时间，但不能往下执行代码。调用非阻塞型io的应用程序，(以前)需要向操作系统轮询文件符是否完成。轮询文件符 === 轮询io操作是否完成。调用非阻塞型io的应用程序忙于轮询，也无法使用cpu时间。

非阻塞型io中read最原始，poll和select类似，epoll最先进。调用epoll io的应用程序也无法拿到cpu时间，不会无效占用cpu时间。epoll，实现了事件通知执行回调的机制。

node中的异步io效果：线程池中取一个线程，调用操作系统的阻塞型io或epoll io。当io完成，将数据返回给父线程。

nodejs在window直接使用IOCP，在linux下epoll API自己实现了基于线程池的异步io，在libuv层完成兼容。window提供了“线程池+阻塞IO”的异步io，称之为IOCP。

node的事件循环，每次循环(tick)查看是否有事件，有事件，则取出事件，再查看是否有关联回调，就有执行它们。

核心模块调内建模块，内建模块调内核API。

以给cpu输入文件为例，fs.open调libuv的uv_fs_open函数，uv_fs_open函数以FSReqWrap为参数调用操作系统的QueueUserWorkItem，调用QueueUserWorkItem后fs.open立即返回，js继续执行后续代码。而交给操作系统的io则由线程池中的线程执行，互相无约束达到异步的效果。

FSReqWrap封装了文件io的api/实参/回调，io api阻塞型、非阻塞型都可以。

简而言之，就是将io交由IOCP，IOCP会利用已有线程，执行io。io完成后，该线程将结果放入FSReqWrap对象并通知IOCP(通过PostQueuedCompletionStatus)。

Node每次tick时都获取IOCP中已完成的io（通过GetQueuedCompletionStatus），将io对应的事件加入自己的io观察者队列，然后取出其回调执行，如图p59。

node中存在一些与IO操作无关的异步API，这些异步API好像是阻塞的。包含setTimeout/setInterval和setImmediate/process.nextTick

非阻塞 ===  并行。非阻塞IO是指与其他的IO操作，与其他的CPU操作可以同时做。

调用setTimeout/setInterval会new出定时器对象，是定时器观察者在管理这些对象。定时器观察者内部维护一颗红黑树。每次Tick执行时，从红黑树迭代出定时器对象，判断是否超过定时时间，如果超过，就产生一个事件并就地执行回调。★★★如果上一个Tick执行时长过长，则该定时器回调无法准时执行

process.nextTick比setTimeout(0)轻量，因为前者只使用队列数组，而后者创建定时器对象使用红黑树都更消耗性能。队列内的回调会在下次Tick时全部执行。

每次Tick中会对不同观察者进行检查。检查是由先后次序的。idle观察者先于IO观察者先于check观察者。process.nextTick时idle观察者，优先级最高。setImmediate是check观察者，使用的是链表，每个Tick只执行一个链表首部的回调。★★★所以，process.nextTick是微任务？setImmediate是宏任务，宏任务是较重的任务所以每次只执行一个。

node监听IO请求初始化结束后，就是处理IO回调了。就像chrome监听click事件初始化结束后，就是处理click回调了。

node的事件循环eventLoop === 在Tick中触发事件

有异步IO，必有异步编程

事件回调机制是基于高阶函数这一基础设施来完成的

可以批量创建函数的函数，如工厂一样创建函数的函数，称为偏函数

可以在node中利用C++拓展调用内核API

在分布式中，各个单点之间保持并行非阻塞，在这样的组织方式下，执行任务速度更快。

事件循环的这种调度方式，是在Tick中请求IO操作/处理IO操作返回的结果，在线程池中执行IO操作。Tick是CPU密集型的，如果Tick过于忙碌，无法及时请求IO操作而线程池中线程闲置，造成CPU和IO操作同步/阻塞，那么整体的处理效率降低。建议一次Tick中CPU计算量不超过10ms，否则就使用setImmediate对CPU计算进行分片。

trycatch只能捕获当前事件循环/Tick中的异常

可以说，setTimeout就是sleep函数。

cluster模块基于child_process模块 

事件监听/发布订阅模式是一种广泛存在于异步编程的模式，是回调函数的“事件化”。

hook/钩子函数，hook的是内部状态。

试试emitter.setMaxListener(0)将监听函数数量限制解除。

对于emitter对象，如果error事件被监听了，则遇到报错是执行该事件回调。没有监听该事件才抛出异常。我们应该监听error事件，提高node线程健壮性。

雪崩问题指当高访问高并发时期缓存失效，大量请求涌入数据库，超过数据库限值。比如在高访问高并发时期启动程序，此时缓存为空。

事件，函数在某确定时机执行，emitter.once('whenResponse'，action)API，场景有使同一sql语句同一时间只进行一个请求，等到这个请求成功时emitter.emit('whenResponse')也让其他同样sql请求成功。emitter.emit('whenResponse')埋点，指定实际执行时机。emitter.once('whenResponse'，action)是指定名义上的执行时机。

emit和on可以不一一对应，一个on对应多个emit，可以在on中为函数定义在多时机同时满足下执行。emit只是所有埋点中的一个。

error函数，在error时机时执行，所以，通过emitter.emit('error')在error时埋点。

事件 === 埋点执行的机制。

Promise所代表的那种模式在jQuery时代就被广为使用:
$.get('/api') // 把异步操作封装成一个异步操作对象/Deferred，这个对象内含一个事件对象/Promise，可以在这个内含对象上定义事件，通过暴露 success API 或 then API。
.success()
.success()
.error()
.complete()

Promise模式从2009年开始被抽象出来，陆续有A，B，D草案提交给CommonJS规范，使得异步IO操作的代码书写可以优雅。

```
const Promise = function(){
    EventEmitter.call(this)
}

```

$.get('/api').success或promise.then方法中为异步操作（对象）定义事件：
```
this.once('whenSuccess',fulfillFunc)
```

Deferred拥有状态域和观察者域/promise。原生异步操作顺利结束后，事先与其构成绑定关系的Deferred将状态掰成fulfilled并调用resolve事件。这种绑定关系是通过在原生对象的事件定义中调用Deferred对象的事件来达到的。

Deferred的本质只是一个观察者对象，通过resolve/reject方法让外部能够调用其事件，通过promise属性的then方法让外部能够定义其事件。而原生异步操作API本身也是一个观察者对象，但只允许外部定义其事件。★★promise模式的本质是一个中间层，之前程序是：原本定义在原生对象上的事件，现在定义在promise对象上，原本异步操作结束后，调用原生对象上的事件，现在调用promise对象上的事件。有了中间层就有了想象空间，就可以做一些事情，比如链式promise。通过判断调用事件后的返回值，如果也是一个promise对象，那么接下来的事件定义，转移到这个新的promise对象上。

在最初的promise规范的Deferrred实现中，除了resolve/reject事件还有progress事件。

p88提供了一个Promise.all的实现，具体实现是new一个deferred，维护一个count计数，在传入的每个promise上调用then定义事件：count--，当count === 0时让deferred调用事件，并返回deferred.promise让外部能够定义事件。

promise.resolve调用事件时,逐个执行事件回调队列里的函数，如果回调返回promise对象，则队列里剩下的回调转移给它

next == emit

如果对文件系统进行大量并发调用，操作系统的文件描述符的数量会被瞬间用光。通过queue对象进行过载保护。queue对象初始化时设置最大并发量，并暴露出push方法。当调用push方法时函数入队/调用_next。而_next方法中，如果当前并发量active小于最大并发量limit，调用队列中等待的第一个函数，否则不执行。而一旦有函数结束，在该函数的回调中active--/调用_next。如果调用push时内部队列长度_qlength已满，则直接报错。如果拒绝模式调用push时已到最大并发量，则直接报错，满足实时性方面的需求。

超时的IO操作也会返回，只是不会返回给程序员。在IO操作发出的那一刻会同时开启一个倒数任务，倒数任务先触发回调则意味着超时。

### 第五章 内存控制

程序需要长时间运行，才需要内存控制

电脑的唯二资源CPU和IO，通过编程语言如node利用。

js的所有对象都是堆对象，在heap中生存。试试process.memoryUsage()

node一开始会向内存申请一定的内存，当已申请的内存不够new新的对象时再向内核申请，直到达到极限直接程序崩溃。内存会有最大值是因为内存大小与GC时间正相关。1.5G的堆内存，小的GC一次50ms，一次全量GC需要1s。GC时js线程挂起。node内存最大值是2G不到。

不同对象的生命周期长度有较大差异。node内存大小 = old-space + new-space。老生代空间最大1400MB，新生代空间最大32MB。差了40倍。

新生代空间垃圾回收使用的是Scavenge/Cheney算法，该算法发表再1970年的ACM上。原理是把内存空间分成From/To两个区域(两个semispace空间)。垃圾回收就是检查From中的存活对象复制到To空间，然后From/To空间的角色对换。Scavenge的特点是用空间换时间。更细节的，在一次Cheney垃圾回收的过程中From中的存活对象可能复制到To空间，可能复制到老生代空间。

一个新对象会出生在From空间，经历过一次Scavenge回收/去过一个To空间，第二次经历Scavenge回收时就会被转移到老生代。或者如果Scavenge回收时From空间已经超过4MB则把From的所有对象送往老生代空间。